In [2]:
cd C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam

C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam


C:\Users\jacob\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# cd C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam

SyntaxError: invalid syntax (1786311159.py, line 3)

In [4]:
from squid_lab_quam.components.roots import QuAMSCQ1

machine = QuAMSCQ1()

In [5]:
from squid_lab_quam.components.network import OPXNetwork, OctaveNetwork

network = OPXNetwork(
    host="10.209.67.168",
    cluster_name="Cluster_1",
    octave_networks={
        "octave1": OctaveNetwork(
            octave_host="10.209.67.174", octave_port=80, controller="con1"
        )
    },
)

machine.network = network

In [6]:
from squid_lab_quam.components.information import Information

information = Information(
    user_name="Jacob Hastrup",
    user_ku_tag="drn455",
    device_name="Chalmers/5q4_v0",
    fridge_name="meso",
    data_path=r"\\unicph.domain\groupdir\SCI-NBI-QDev\SQuID Lab Data\Devices\Chalmers 5q4c v1.1\OPX Data",
    state_path=r"C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests",
)

machine.information = information

information.print_info()

User name: Jacob Hastrup (drn455)
Device name: Chalmers/5q4_v0
Fridge name: meso


In [7]:
from squid_lab_quam.components.wiring import (
    OPXWiring,
    OPXFeedLineWiring,
    OPXIQChannelWiring,
)

wiring = OPXWiring(
    drive_lines={
        "q0": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q1": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q2": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q3": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q4": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
    },
    feed_lines={
        "feedline": OPXFeedLineWiring(
            output_I=("con1", 0),
            output_Q=("con1", 0),
            input_I=("con1", 0),
            input_Q=("con1", 0),
        )
    },
)

machine.wiring = wiring

In [8]:
from squid_lab_quam.components.octave import OctaveSQuID

octaves = {
    "octave1": OctaveSQuID(
        ip=network.octave_networks["octave1"].get_reference("octave_host"),
        port=network.octave_networks["octave1"].get_reference("octave_port"),
    )
}

machine.octaves = octaves

In [11]:
from squid_lab_quam.components.qubits import Transmon
from squid_lab_quam.components.octave import OctaveUpConverterSQuID, OctaveSQuID
from quam.components.channels import IQChannel

qubit_names = ["q0", "q1", "q2", "q3", "q4"]

qubits = {}

for idx, qubit in enumerate(qubit_names):
    # Add driving frequency converter
    drive_up_converter = OctaveUpConverterSQuID(
    )
    octaves["octave1"].RF_outputs[idx + 2] = drive_up_converter


    qubits[qubit] = Transmon(
        xy=IQChannel(
            opx_output_I=f"#/wiring/drive_lines/{qubit}/port_I",
            opx_output_Q=f"#/wiring/drive_lines/{qubit}/port_Q",
            frequency_converter_up=drive_up_converter.get_reference()
        )
    )

machine.qubits = qubits